# Driverless AI + MLFlow

![Integration Diagram](files/tables/DB_diag.png)

In [0]:
import datatable as dt
import daimojo.model
import pandas as pd
import os

In [0]:
import os
import shutil
from sys import version_info
import argparse
import cloudpickle
import mlflow.pyfunc
def get_env():
    python_version = "{major}.{minor}.{micro}".format(
        major=version_info.major,
        minor=version_info.minor,
        micro=version_info.micro,
    )
    daimojo_path = (
        "http://artifacts.h2o.ai.s3.amazonaws.com/releases/ai/h2o/daimojo/2.5.8/x86_64-centos7/daimojo-2.5.8-cp37-cp37m-linux_x86_64.whl"
    )
    conda_env = {
        "channels": ["defaults"],
        "dependencies": [
            "python={}".format(python_version),
            "pip",
            {
                "pip": [
                    "mlflow",
                    "datatable",
                    "--no-binary=protobuf protobuf",
                    daimojo_path,
                    "cloudpickle=={}".format(cloudpickle.__version__),
                ],
            },
        ],
        "name": "dai_env",
    }
    return conda_env
def get_artifacts(mojo_path, license_path, feature_names):
    artifacts = {
        "dai_model": mojo_path,
        "license_key": license_path,
        "feature_names": feature_names
    }
    return artifacts
class DAIWrappedModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        import daimojo.model
        import os
        self.model = daimojo.model(context.artifacts["dai_model"])
        license_file = context.artifacts["license_key"]
        os.environ["DRIVERLESS_AI_LICENSE_FIlE"] = license_file
        with open(context.artifacts["feature_names"], "r") as file:
          self.feature_names = file.read().split(",")
    def predict(self, context, x): #test
        import datatable as dt
        #import os
        #license_file = context.artifacts["license_key"]
        #os.environ["DRIVERLESS_AI_LICENSE_FIlE"] = license_file
        x.columns = self.feature_names
        pred = self.model.predict(dt.Frame(x))
        if pred.shape[1] == 2:
            pred = pred[:, 1]
        return pred.to_pandas()
def log_model(model_path, mojo_path, license_path, feature_names):
    conda_env = get_env()
    artifacts = get_artifacts(mojo_path, license_path, feature_names)
    mlflow.pyfunc.log_model(
        artifact_path=model_path,
        python_model=DAIWrappedModel(),
        artifacts=artifacts,
        conda_env=conda_env,
    )      

In [0]:
dbutils.fs.cp("/FileStore/tables/pipeline.mojo","file:///databricks/pipeline.mojo")
dbutils.fs.cp("/FileStore/tables/license.sig","file:///databricks/license.sig")
dbutils.fs.cp("/FileStore/tables/example.csv","file:///databricks/example.csv")

Out[39]: True

In [0]:
# Specify license file location
os.environ["DRIVERLESS_AI_LICENSE_FILE"]= "/databricks/license.sig"

## Data Prep in Spark/Delta

In [0]:
# read in data
# add column
# split for training
# update delta table

In [0]:
train_df = spark.read.csv('s3://h2o-public-test-data/smalldata/kaggle/CreditCard/creditcard_train_cat.csv', header = True, inferSchema = True)
test_df = spark.read.csv('s3://h2o-public-test-data/smalldata/kaggle/CreditCard/creditcard_test_cat.csv', header = True, inferSchema = True)


## Train model in DAI

In [0]:
# connect to DAI instance

import driverlessai

address = WAVE_DAI_ADDRESS
username = WAVE_DAI_USERNAME
password = WAVE_DAI_PASSWORD
dai = driverlessai.Client(address = address, username = username, password = password)

In [0]:
# Send data to DAI from S3 path

train_path = WAVE_TRAIN_FILE_PATH
test_path = WAVE_TEST_FILE_PATH

train = dai.datasets.create(data=train_path, data_source='s3')
test = dai.datasets.create(data=test_path, data_source='s3')

--------------------------------------------------------------------------- 
 DatasetExists Traceback (most recent call last)
 <command-1272602360004180> in <module> 
 4 test_path = 's3://h2o-public-test-data/smalldata/kaggle/CreditCard/creditcard_test_cat.csv' 
 5 
 ----> 6 train = dai . datasets . create ( data = train_path , data_source = 's3' ) 
 7 test = dai . datasets . create ( data = test_path , data_source = 's3' ) 

 /databricks/python/lib/python3.7/site-packages/driverlessai/_datasets.py in create (self, data, data_source, data_source_config, force, name) 
 1148 """
 1149 return self.create_async(
 -> 1150 data , data_source , data_source_config , force , name
 1151 ).result()
 1152 

 /databricks/python/lib/python3.7/site-packages/driverlessai/_datasets.py in create_async (self, data, data_source, data_source_config, force, name) 
 1179 )
 1180 """
 -> 1181 return self . _dataset_create ( data , data_source , data_source_config , force , name ) 
 1182 
 1183 def get ( self , key : str ) -> "Dataset" : 

 /databricks/python/lib/python3.7/site-packages/driverlessai/_datasets.py in _dataset_create (self, data, data_source, data_source_config, force, name) 
 1031 elif isinstance ( data , str ) : 
 1032 # if data is not Pandas DataFrame 
 -> 1033 _utils . error_if_dataset_exists ( self . _client , Path ( data ) . name ) 
 1034 if data_source in self . _simple_connectors : 
 1035 key = self . _simple_connectors [ data_source ] ( data ) 

 /databricks/python/lib/python3.7/site-packages/driverlessai/_utils.py in error_if_dataset_exists (client, name) 
 401 if name in client . _backend . list_datasets_with_similar_name ( name ) : 
 402 raise DatasetExists(
 --> 403 f"Dataset with name '{name}' already exists on server. " 
 404 "Use `force=True` to create another dataset with same name." 
 405 )

 DatasetExists : Dataset with name 'creditcard_train_cat.csv' already exists on server. Use `force=True` to create another dataset with same name.

In [0]:
dai.datasets.list()

Out[21]: [<class 'Dataset'> b2b44b0e-7069-11eb-964d-0242ac110002 creditcard_test_cat.csv,
 <class 'Dataset'> b0f1ee34-7069-11eb-964d-0242ac110002 creditcard_train_cat.csv,
 <class 'Dataset'> c687f37e-64c5-11eb-8877-0242ac110002 lend_test,
 <class 'Dataset'> c687cbf6-64c5-11eb-8877-0242ac110002 lend_train,
 <class 'Dataset'> d2fc9660-64c4-11eb-8877-0242ac110002 CreditCard-test.csv,
 <class 'Dataset'> d2fc0970-64c4-11eb-8877-0242ac110002 CreditCard-train.csv,
 <class 'Dataset'> da291cdc-3a3b-11eb-9c9f-0242ac110002 malware_test,
 <class 'Dataset'> da28f4be-3a3b-11eb-9c9f-0242ac110002 malware_train,
 <class 'Dataset'> c1aa5c70-3a3b-11eb-9c9f-0242ac110002 network-traffic-android-malware.zip,
 <class 'Dataset'> df83c698-2846-11eb-bf1d-0242ac110002 walmart_test.csv,
 <class 'Dataset'> df8377b0-2846-11eb-bf1d-0242ac110002 walmart_train.csv,
 <class 'Dataset'> 48331a3e-245c-11eb-9abd-0242ac110002 GF_rolling_mean_25,
 <class 'Dataset'> a5290a60-245b-11eb-9abd-0242ac110002 GF_demo,
 <class 'Dataset'> 7b5bebba-2426-11eb-8817-0242ac110002 GF_rolling_means25_2,
 <class 'Dataset'> aa71a026-2425-11eb-8817-0242ac110002 GF_rolling_mean25_1,
 <class 'Dataset'> 91f4db58-22c7-11eb-8048-0242ac110002 ExampleFileForML_Page01.txt,
 <class 'Dataset'> 825bfea6-22c7-11eb-8048-0242ac110002 ExampleFileForML_Page02.txt,
 <class 'Dataset'> e25f557e-1ee8-11eb-be57-0242ac110002 Lending_club_test.csv,
 <class 'Dataset'> 58cd8aa0-1ee4-11eb-be57-0242ac110002 Lending_club_test_good_no_label.csv,
 <class 'Dataset'> 12cf5dbc-1ee4-11eb-be57-0242ac110002 Lending_club_test_no_label.csv,
 <class 'Dataset'> 15e4a854-1edf-11eb-be57-0242ac110002 Lending_club_train.csv,
 <class 'Dataset'> 1272db6e-1edf-11eb-be57-0242ac110002 Lending_club_train_good.csv,
 <class 'Dataset'> 786377b0-19eb-11eb-86f1-0242ac110002 CreditCard-test.csv,
 <class 'Dataset'> 7724159e-19eb-11eb-86f1-0242ac110002 CreditCard-train.csv,
 <class 'Dataset'> d917a98e-19df-11eb-86f1-0242ac110002 AAPL_test,
 <class 'Dataset'> d9177d42-19df-11eb-86f1-0242ac110002 AAPL_train,
 <class 'Dataset'> affb7602-19df-11eb-86f1-0242ac110002 AAPL_stock.csv,
 <class 'Dataset'> 5f156554-19df-11eb-86f1-0242ac110002 all_stocks_5yr.csv,
 <class 'Dataset'> 6a2fa552-19db-11eb-86f1-0242ac110002 test_data.csv,
 <class 'Dataset'> 68eb2c52-19db-11eb-86f1-0242ac110002 train_data.csv]

In [0]:
#dai.datasets

train = dai.datasets.list()[1]
test = dai.datasets.list()[0]

In [0]:
# set up experiment

In [0]:
dai_target = WAVE_DAI_TARGET
dai_acc = WAVE_DAI_ACC
dai_time = WAVE_DAI_TIME
dai_int = WAVE_DAI_INTERPRETABILITY
dai_scorer = WAVE_DAI_SCORER
dai_cols_to_drop = WAVE_DAI_COLS2DROP
exp_preview = dai.experiments.preview(train_dataset=train,
                                      task='classification',
                                      target_column=dai_target,
                                      enable_gpus=False,
                                      accuracy=dai_acc,
                                      time=dai_time,
                                      interpretability=dai_int,
                                      config_overrides=None)
exp_preview

ACCURACY [5/10]:
- Training data size: *23,999 rows, 25 cols*
- Feature evolution: *[Constant, LightGBM, XGBoostGBM]*, *1/4 validation split*
- Final pipeline: *Blend of up to 2 [Constant, LightGBM, XGBoostGBM] models, each averaged across 4-fold CV splits*

TIME [5/10]:
- Feature evolution: *4 individuals*, up to *66 iterations*
- Early stopping: After *10* iterations of no improvement

INTERPRETABILITY [5/10]:
- Feature pre-pruning strategy: None
- Monotonicity constraints: disabled
- Feature engineering search space: [CVCatNumEncode, CVTargetEncode, CatOriginal, Cat, ClusterDist, ClusterTE, Frequent, Interactions, NumCatTE, NumToCatTE, NumToCatWoE, OneHotEncoding, Original, TextLinModel, TextOriginal, Text, TruncSVDNum, WeightOfEvidence]

[Constant, LightGBM, XGBoostGBM] models to train:
- Model and feature tuning: *16*
- Feature evolution: *104*
- Final pipeline: *8*

Estimated runtime: *minutes*
Auto-click Finish/Abort if not done in: *1 day*/*7 days*

In [0]:
# run experiment

In [0]:
ex = dai.experiments.create(train_dataset=train,
                            test_dataset=test,
                            target_column=dai_target,
                            task='classification',
                            accuracy=dai_acc,
                            time=dai_time,
                            interpretability=dai_int,
                            scorer=dai_scorer,
                            enable_gpus=True,
                            seed=1234,
                            cols_to_drop=dai_cols_to_drop,
                            name = "credit_mlflow")

Experiment launched at: http://ec2-3-82-197-13.compute-1.amazonaws.com/#/experiment?key=537d59a4-7204-11eb-bc58-0242ac110002
Running 0.00% - Ingesting training data... Running 0.00% - Ingesting test data... Running 0.00% - Detecting column types... Running 0.00% - Detecting column types... Running 0.00% - Tuning modeling backend... Running 0.00% - Tuning modeling backend... Running 0.00% - Tuning modeling backend... Running 0.00% - Tuning modeling backend... Running 0.00% - Checking for data shift... Running 0.00% - Checking for data shift... Running 0.00% - Checked shift: 8/23 Running 0.00% - Checked shift: 16/23 Running 0.00% - Checking for leakage... Running 0.00% - Model and feature tuning... Running 0.00% - Model and feature tuning... Running 0.00% - Model and feature tuning... Running 0.00% - Model and feature tuning... Running 0.00% - Model and feature tuning... Running 0.00% - Tuned 0/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 0.00% - Tuned 0/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 1.28% - Tuned 0/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 1.28% - Tuned 0/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 1.28% - Tuned 0/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 1.28% - Tuned 0/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 1.91% - Tuned 1/18 Parameter & feature tuning models. Tuning [GLM] Running 1.91% - Tuned 1/18 Parameter & feature tuning models. Tuning [GLM] Running 1.91% - Tuned 1/18 Parameter & feature tuning models. Tuning [GLM] Running 1.91% - Tuned 1/18 Parameter & feature tuning models. Tuning [GLM] Running 2.55% - Tuned 2/18 Parameter & feature tuning models. Tuning [XGBOOSTGBM] Running 2.55% - Tuned 2/18 Parameter & feature tuning models. Tuning [XGBOOSTGBM] Running 2.55% - Tuned 2/18 Parameter & feature tuning models. Tuning [XGBOOSTGBM] Running 3.19% - Tuned 3/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 3.19% - Tuned 3/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 3.19% - Tuned 3/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 3.83% - Tuned 4/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 3.83% - Tuned 4/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 3.83% - Tuned 4/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 4.47% - Tuned 5/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 4.47% - Tuned 5/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 4.47% - Tuned 5/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 5.11% - Tuned 6/18 Parameter & feature tuning models. Tuning [GLM] Running 5.11% - Tuned 6/18 Parameter & feature tuning models. Tuning [GLM] Running 5.11% - Tuned 6/18 Parameter & feature tuning models. Tuning [GLM] Running 5.74% - Tuned 7/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 5.74% - Tuned 7/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 5.74% - Tuned 7/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 5.74% - Tuned 7/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 6.38% - Tuned 8/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 6.38% - Tuned 8/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 7.02% - Tuned 9/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 7.02% - Tuned 9/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 7.02% - Tuned 9/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 7.02% - Tuned 9/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 7.02% - Tuned 9/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 7.66% - Tuned 10/18 Parameter & feature tuning models. Tuning [LIGHTGBM] Running 7.66% - Tuned 10/18 Parameter & feature tuning models. Tuning [LIG

In [0]:
dai.experiments.list()

Out[26]: [<class 'Experiment'> 537d59a4-7204-11eb-bc58-0242ac110002 credit_mlflow,
 <class 'Experiment'> aca0a57a-7134-11eb-969a-0242ac110002 panifute,
 <class 'Experiment'> d66e9da6-7069-11eb-964d-0242ac110002 bawukepu,
 <class 'Experiment'> b82035ae-6c9e-11eb-a0f0-0242ac110002 hipatopi,
 <class 'Experiment'> e6aa3550-64ce-11eb-8877-0242ac110002 1.gosoruge,
 <class 'Experiment'> 9d171ef2-64ca-11eb-8877-0242ac110002 hubucalo,
 <class 'Experiment'> 57db5868-3fa3-11eb-bcbd-0242ac110002 gosoruge,
 <class 'Experiment'> 88a04ce2-3edb-11eb-a9ac-0242ac110002 Single TensorFlow-kitevobu,
 <class 'Experiment'> 879893a4-3edb-11eb-a9ac-0242ac110002 Single FTRL-kitevobu,
 <class 'Experiment'> 869d2294-3edb-11eb-a9ac-0242ac110002 Default Single LightGBM-kitevobu,
 <class 'Experiment'> 85adb4c0-3edb-11eb-a9ac-0242ac110002 Few Features Single LightGBM-kitevobu,
 <class 'Experiment'> 84c295ee-3edb-11eb-a9ac-0242ac110002 Complex LightGBM Ensemble-kitevobu,
 <class 'Experiment'> 83d9b18a-3edb-11eb-a9ac-0242ac110002 Default Settings-kitevobu,
 <class 'Experiment'> 82f4b986-3edb-11eb-a9ac-0242ac110002 Single GLM-kitevobu,
 <class 'Experiment'> 8212c26a-3edb-11eb-a9ac-0242ac110002 Single DecisionTree-kitevobu,
 <class 'Experiment'> 813573ce-3edb-11eb-a9ac-0242ac110002 Constant Baseline-kitevobu,
 <class 'Experiment'> 80585656-3edb-11eb-a9ac-0242ac110002 Simple LightGBM-kitevobu,
 <class 'Experiment'> 7f79d1f6-3edb-11eb-a9ac-0242ac110002 Few Features DecisionTree-kitevobu,
 <class 'Experiment'> 15cb91fa-3eda-11eb-a9ac-0242ac110002 dutekuba,
 <class 'Experiment'> 48e90ed4-3a46-11eb-9c9f-0242ac110002 lenitewi,
 <class 'Experiment'> eeccf0f2-3a3e-11eb-9c9f-0242ac110002 Single TensorFlow-Malware_leader,
 <class 'Experiment'> ee03c9b6-3a3e-11eb-9c9f-0242ac110002 Single FTRL-Malware_leader,
 <class 'Experiment'> ed3d718a-3a3e-11eb-9c9f-0242ac110002 Default Single LightGBM-Malware_leader,
 <class 'Experiment'> ec7ca144-3a3e-11eb-9c9f-0242ac110002 Few Features Single LightGBM-Malware_leader,
 <class 'Experiment'> ebbee3b6-3a3e-11eb-9c9f-0242ac110002 Complex LightGBM Ensemble-Malware_leader,
 <class 'Experiment'> eb021826-3a3e-11eb-9c9f-0242ac110002 Default Settings-Malware_leader,
 <class 'Experiment'> ea49e6f2-3a3e-11eb-9c9f-0242ac110002 Single GLM-Malware_leader,
 <class 'Experiment'> e99566a0-3a3e-11eb-9c9f-0242ac110002 Single DecisionTree-Malware_leader,
 <class 'Experiment'> e8e606ba-3a3e-11eb-9c9f-0242ac110002 Constant Baseline-Malware_leader,
 <class 'Experiment'> e83987dc-3a3e-11eb-9c9f-0242ac110002 Simple LightGBM-Malware_leader,
 <class 'Experiment'> e7907f66-3a3e-11eb-9c9f-0242ac110002 Few Features DecisionTree-Malware_leader,
 <class 'Experiment'> 725d69e4-3a3d-11eb-9c9f-0242ac110002 Malware_all,
 <class 'Experiment'> 5c58e0ee-359a-11eb-9c9c-0242ac110002 1.IID_manual_lags_no_date,
 <class 'Experiment'> 84fa77bc-28fa-11eb-8008-0242ac110002 kesaheme,
 <class 'Experiment'> 8fe940e4-284c-11eb-bf1d-0242ac110002 kekinese,
 <class 'Experiment'> c681abde-2499-11eb-8a39-0242ac110002 Single TensorFlow-1.gadapitu,
 <class 'Experiment'> c5fa805a-2499-11eb-8a39-0242ac110002 Single FTRL-1.gadapitu,
 <class 'Experiment'> c5764d1c-2499-11eb-8a39-0242ac110002 Default Single LightGBM-1.gadapitu,
 <class 'Experiment'> c4ca985a-2499-11eb-8a39-0242ac110002 Few Features Single LightGBM-1.gadapitu,
 <class 'Experiment'> c4061958-2499-11eb-8a39-0242ac110002 Complex LightGBM Ensemble-1.gadapitu,
 <class 'Experiment'> c372c108-2499-11eb-8a39-0242ac110002 Default Settings-1.gadapitu,
 <class 'Experiment'> c2f78cd6-2499-11eb-8a39-0242ac110002 Single GLM-1.gadapitu,
 <class 'Experiment'> c2849be0-2499-11eb-8a39-0242ac110002 Single DecisionTree-1.gadapitu,
 <class 'Experiment'> c2152abc-2499-11eb-8a39-0242ac110002 Constant Baseline-1.gadapitu,
 <class 'Experiment'> c1a8fe14-2499-11eb-8a39-0242ac110002 Simple LightGBM-1.gadapitu,
 <class 'Experiment'> c13f8448-2499-11eb-8a39-0242ac110002 Few Features DecisionTree-1.gadapitu,
 <class 'Experiment'> 572a22ac-2471-11eb-9abd-0242a

In [0]:
ex = [ex for ex in dai.experiments.list() if 'credit' in ex.name][0]

In [0]:
# get model results

In [0]:
metrics = ex.metrics()

print("Final model Score on Validation Data: " + str(round(metrics['val_score'], 3)))

Final model Score on Validation Data: 0.776

In [0]:
ex.summary()

Status: Complete
Experiment: credit_mlflow (537d59a4-7204-11eb-bc58-0242ac110002)
 Version: 1.9.0.2, 2021-02-18 16:19
 Settings: 5/4/6, seed=606722497, GPUs disabled
 Train data: creditcard_train_cat.csv (23999, 25)
 Validation data: N/A
 Test data: [Test] (6000, 24)
 Target column: DEFAULT_PAYMENT_NEXT_MONTH (binary, 22.372% target class)
System specs: Docker/Linux, 62 GB, 16 CPU cores, 0/0 GPU
 Max memory usage: 0.667 GB, 0 GB GPU
Recipe: AutoDL (23 iterations, 4 individuals)
 Validation scheme: stratified, 1 internal holdout
 Feature engineering: 223 features scored (22 selected)
Timing: MOJO latency: 0.07447 millis (902.7KB)
 Data preparation: 8.21 secs
 Shift/Leakage detection: 6.58 secs
 Model and feature tuning: 107.29 secs (18 models trained)
 Feature evolution: 72.14 secs (11 of 84 models trained)
 Final pipeline training: 40.89 secs (8 models trained)
 Python / MOJO scorer building: 77.50 secs / 9.69 secs
Validation score: AUC = 0.5 (constant preds of 0.2237)
Validation score: AUC = 0.7820838 +/- 0.006205764 (baseline)
Validation score: AUC = 0.776198 +/- 0.007268345 (final pipeline)
Test score: AUC = N/A (no target)

In [0]:
print(ex.variable_importance())

gain | interaction | description
------------+---------------------+-----------------------------------------------------------------------------------------------------------------------
 1 | 8_PAY_1 | PAY_1 (Original)
 0.277236 | 9_PAY_2 | PAY_2 (Original)
 0.123436 | 1_BILL_AMT1 | BILL_AMT1 (Original)
 0.122974 | 7_LIMIT_BAL | LIMIT_BAL (Original)
 0.117768 | 15_PAY_AMT2 | PAY_AMT2 (Original)
 0.0922378 | 14_PAY_AMT1 | PAY_AMT1 (Original)
 0.0909016 | 17_PAY_AMT4 | PAY_AMT4 (Original)
 0.0879165 | 16_PAY_AMT3 | PAY_AMT3 (Original)
 0.0758115 | 0_AGE | AGE (Original)
 0.0735265 | 2_BILL_AMT2 | BILL_AMT2 (Original)
 0.0677384 | 19_PAY_AMT6 | PAY_AMT6 (Original)
 0.0646439 | 10_PAY_3 | PAY_3 (Original)
 0.0628528 | 11_PAY_4 | PAY_4 (Original)
 0.061981 | 3_BILL_AMT3 | BILL_AMT3 (Original)
 0.0580812 | 18_PAY_AMT5 | PAY_AMT5 (Original)
 0.0579036 | 29_CVTE:EDUCATION.0 | Out-of-fold mean of the response grouped by: ['EDUCATION'] using 5 folds [internal:(10, 3, 10)]
 0.0568053 | 4_BILL_AMT4 | BILL_AMT4 (Original)
 0.0552498 | 6_BILL_AMT6 | BILL_AMT6 (Original)
 0.0510243 | 5_BILL_AMT5 | BILL_AMT5 (Original)
 0.0474112 | 13_PAY_6 | PAY_6 (Original)
 0.0432579 | 28_CVTE:MARRIAGE.0 | Out-of-fold mean of the response grouped by: ['MARRIAGE'] using 5 folds [internal:(10, 3, 10)]
 0.00912965 | 30_Freq:EDUCATION | Encoding of categorical levels of feature(s) ['EDUCATION'] to value between 0 and 1 based on their relative frequency

## Log model and metrics in MLFlow

In [0]:
# log model artifact

In [0]:
ex.artifacts.create('mojo_pipeline')

Building MOJO pipeline...

In [0]:
ex.artifacts.download(only='mojo_pipeline', dst_dir='/dbfs/FileStore/', overwrite=True)

Downloaded '/dbfs/FileStore/mojo.zip'
Out[33]: {'mojo_pipeline': '/dbfs/FileStore/mojo.zip'}

In [0]:
%sh

unzip /dbfs/FileStore/mojo.zip -d /dbfs/FileStore/


Archive: /dbfs/FileStore/mojo.zip

In [0]:
with open("/databricks/feature_names.txt", "w") as file: 
  file.write(",".join(test_df.columns))

In [0]:
log_model("DAI_test3", "dbfs:/FileStore/mojo-pipeline/pipeline.mojo", "/databricks/license.sig", "/databricks/feature_names.txt")

In [0]:
metrics

Out[44]: {'scorer': 'AUC',
 'val_score': 0.7761980435194669,
 'val_score_sd': 0.0072683448678199205,
 'val_roc_auc': 0.7761980435194669,
 'val_pr_auc': 0.5527748720450862,
 'test_score': None,
 'test_score_sd': None,
 'test_roc_auc': None,
 'test_pr_auc': None}

In [0]:
# log model metrics

import mlflow

mlflow.log_params(metrics)

In [0]:
# leverage autodoc artifcats for tracking more metrics

##Get 
# pipeline.mojo
#Four files from summary for tracking:
# ensemble_model_description.json
# ensemble_scores.json
# features_orig.json
# features.json

##Track:
#input feature names
#original feature importance
#transformed feature importance
#Ensemble model agorithms, weights, and hyperparameters for each
#Ensemble model performance on training and test sets




In [0]:
# add experiment to model registry

In [0]:
# view experiment in mlflow

## Score new data

In [0]:
# Load Test Data
test_df = spark.read.csv(WAVE_TEST_FILE_PATH, header = True, inferSchema = True)

In [0]:
# score with MLFlow
import mlflow
logged_model = 'dbfs:/databricks/mlflow-tracking/1272602360004101/16bb39ce9ebf4390bafa6fcfa5a9eb25/artifacts/DAI_test3'

# Load model as a Spark UDF.
loaded_model = mlflow.pyfunc.spark_udf(spark, logged_model)

features = test_df.columns

# Predict on a Spark DataFrame.
pred = test_df.withColumn('my_predictions', loaded_model(*features))

In [0]:
display(pred)

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,my_predictions
24001,50000,male,university,single,23,2,2,0,0,0,0,51246,49758,48456,44116,21247,20066,8,2401,2254,2004,704,707,0.615347370505333
24002,60000,male,university,single,26,0,0,0,0,0,0,58072,59040,57416,55736,26958,28847,2282,2324,2049,2000,3000,1120,0.16263717040419579
24003,400000,male,university,single,27,0,0,0,0,0,0,15330,8626,11470,10745,20737,9545,2501,10009,1437,1105,510,959,0.06912195961922407
24004,20000,male,other,single,27,5,4,3,2,2,2,21673,21051,20440,19709,20113,19840,0,0,0,900,0,0,0.6552194207906723
24005,50000,male,highschool,single,27,0,0,-2,-2,-1,-1,32590,-100,0,0,70,120,0,100,0,70,200,100,0.12219851650297642
24006,110000,male,university,single,27,0,0,0,0,0,0,102551,103550,105089,107164,105988,108617,5500,6000,6000,4000,5000,4000,0.08726406842470169
24007,30000,male,highschool,single,23,0,0,-2,-1,0,0,4443,370,380,590,7704,20204,430,400,601,7504,15005,5674,0.11338695324957371
24008,230000,male,university,single,27,0,0,0,0,0,0,34592,23689,27652,8430,9811,9865,1816,5105,1293,2000,528,3000,0.06829644925892353
24009,20000,male,highschool,divorce,23,0,0,0,0,0,0,18455,19990,20215,19298,19807,12294,2000,20000,1612,1121,702,1000,0.10288025997579098
24010,30000,male,graduate,single,24,0,0,0,2,0,0,6003,4912,6214,3311,4430,906,1440,2259,0,1500,425,895,0.24969055503606796


In [0]:
# score new data with Rest Server 
# kick off Megan's model monitoring and reporting (drift detection etc.) (where to store and show? MLFlow model registry?)

In [0]:
# score new data with DB scorer
# kick off Megan's model monitoring and reporting (drift detection etc.)

In [0]:
# update delta table

In [0]:
loaded_model

Out[43]: mlflow.pyfunc.loaded_model:
 artifact_path: DAI_test1
 flavor: mlflow.pyfunc.model
 run_id: 4916e854b7f549679620226c024468cd

In [0]:
#TO DO
#can we use feature names from MOJO to pass to Spark UDF? instead of writing to a feature_names file
#provide license.sig to every worker without setting an env variable at the cluster level

#use the DAI model name when calling the model - mlflow experiment tracking
#from MOJO can track features, DAI version, tree hyperparamters
#need to track performance metrics - when training easy, but when scoring?

#full circle: push DAI model to mlflow, generate tracking ID, then load model into Databricks using the tracking ID and score 
#auto update experiments - which mlflow to push it too?

#can we transform, not just score the data on spark?

In [0]:
#workflow

#data manipulation/prep/ETL in DB
#send data to DAI for model training with Python client (extend driverlessai to include this functionality, end user: pip install driverlessai on DB)
#track and log resulting model in MLflow
#for ex mlflow.dai.log_model


In [0]:
#try pip install driverless ai and create experiment
#download artifacts

In [0]:
#data in delta
#data prep using Spark
#send to DAI